In [1]:
import datetime
import math
# !pip install deepeval
# !pip install langchain_community
# !pip install -qU pypdf
# !pip install gigachain-community gigachain-chroma
# !pip install sentence-transformers
# !pip install langsmith
# !pip install pymupdf

import nltk

nltk.download('punkt')
from langchain_chroma import Chroma
from langchain_community.embeddings.gigachat import GigaChatEmbeddings

import os
import re
from langchain_community.document_loaders import PyPDFLoader

[nltk_data] Downloading package punkt to /Users/gd/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
from config import sber
# sber = "NjM5NzdhZmQtZDg0Yy00NmVjLTgyYTctNTViMjYwMGRjMmQ3OjdkMmM0MmQ0LWZkNzItNDczNy1iNjliLTlkYjU2MjE1NWFlYw=="
embedding_function_gigachat = GigaChatEmbeddings(credentials=sber, verify_ssl_certs=False,
                                                 scope="GIGACHAT_API_PERS")

In [3]:
cb_rf_acts = '/Users/gd/PycharmProjects/altirix_systems_chatbot/cb_rf_acts/'
cb_rf_info_letters = '/Users/gd/PycharmProjects/altirix_systems_chatbot/cb_rf_info_letters/'
cb_rf_methodical_recommendations = '/Users/gd/PycharmProjects/altirix_systems_chatbot/cb_rf_methodical_recommendations/'
cb_rf_policies = '/Users/gd/PycharmProjects/altirix_systems_chatbot/cb_rf_policies/'
cb_rf_position = '/Users/gd/PycharmProjects/altirix_systems_chatbot/cb_rf_position/'
cb_rf_prikazy = '/Users/gd/PycharmProjects/altirix_systems_chatbot/cb_rf_prikazy/'
cb_rf_recommendations = '/Users/gd/PycharmProjects/altirix_systems_chatbot/cb_rf_recommendations/'
fsb_documents_pdf = '/Users/gd/PycharmProjects/altirix_systems_chatbot/fsb_documents_pdf/'
fstek_documents_pdf = '/Users/gd/PycharmProjects/altirix_systems_chatbot/fstek_documents_pdf/'
fz = '/Users/gd/PycharmProjects/altirix_systems_chatbot/fz/'
gossopka_acts = '/Users/gd/PycharmProjects/altirix_systems_chatbot/gossopka_acts/'
gosts = '/Users/gd/PycharmProjects/altirix_systems_chatbot/gosts/'
minimum_docs = '/Users/gd/PycharmProjects/altirix_systems_chatbot/minimum_docs/'
# kaspersky_encyclopedia = '/Users/gd/PycharmProjects/altirix_systems_chatbot/kaspersky_encyclopedia/'
# gossopka_glossary = '/Users/gd/PycharmProjects/altirix_systems_chatbot/gossopka_glossary/'

In [4]:
docs_path = ['/Users/gd/Downloads/ФСТЭК_235.pdf']

In [ ]:
for path in sorted(os.listdir(gosts)):
    docs_path.append(gosts + path)

In [ ]:
len(docs_path)

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter


def clean_text(file_path):
    docs = PyPDFLoader(file_path).load()
    for i in range(len(docs)):
        if i == 0:
            docs[i].page_content = re.sub(r'Документ предоставлен.*\n', '', docs[i].page_content, flags=re.DOTALL)
        else:
            docs[i].page_content = re.sub(r'КонсультантПлюс.*\.\.\.', '', docs[i].page_content, flags=re.DOTALL)

    return docs


all_docs = []
for path in docs_path:
    cleaned_docs = clean_text(path)
    all_docs.append(cleaned_docs)


In [6]:
text_splitter = RecursiveCharacterTextSplitter(
    separators=[
        "\n\n",
        "\n",
        " ",
        ".",
        ",",
        "\u200b",
        "\uff0c",
        "\u3001",
        "\uff0e",
        "\u3002",
        "\t",
        ''
    ],
    chunk_size=1200,
    chunk_overlap=70,
    length_function=len
)

In [7]:
for i in range(len(all_docs)):
    for j in range(len(all_docs[i])):
        all_docs[i][j].page_content = all_docs[i][j].page_content.replace('\xad\n', '')
        all_docs[i][j].page_content = all_docs[i][j].page_content.replace(' -\n', '')
        all_docs[i][j].page_content = all_docs[i][j].page_content.replace("-\n", "")
        all_docs[i][j].page_content = all_docs[i][j].page_content.replace("\n", " ")
        all_docs[i][j].page_content = all_docs[i][j].page_content.replace('\u2002', ' ')
        all_docs[i][j].page_content = all_docs[i][j].page_content.replace('\xa0', ' ')
        all_docs[i][j].page_content = all_docs[i][j].page_content.replace('\\u', '')
        all_docs[i][j].page_content = all_docs[i][j].page_content.replace("\t", " ")

In [8]:
chunks = []
for docs in all_docs:
    chunks.append(text_splitter.split_documents(docs))

In [13]:
len([0])

39

In [2]:
import chromadb

persist_directory = "/Users/gd/PycharmProjects/altirix_systems_chatbot/chromadb_chunk_size_1200"
client = chromadb.PersistentClient(path=persist_directory)
client.get_or_create_collection(name="chromadb_chunk_size_1200")

Collection(name=chromadb_chunk_size_1200)

In [63]:
chromadb_chunk_size_1200 = Chroma(
    persist_directory="/Users/gd/PycharmProjects/altirix_systems_chatbot/chromadb_chunk_size_1200_fstek_cosine",
    embedding_function=embedding_function_gigachat)


In [20]:
db_info = chromadb_chunk_size_1200._collection.get(include=["documents", "metadatas", "embeddings"])

In [21]:
print(db_info['ids'])

['doc3544', 'doc3545', 'doc3546', 'doc3547', 'doc3548', 'doc3549', 'doc3550', 'doc3551', 'doc3552', 'doc3553', 'doc3554', 'doc3555', 'doc3556', 'doc3557', 'doc3558', 'doc3559', 'doc3560', 'doc3561', 'doc3562', 'doc3563', 'doc3564', 'doc3565', 'doc3566', 'doc3567', 'doc3568', 'doc3569', 'doc3570', 'doc3571', 'doc3572', 'doc3573', 'doc3574', 'doc3575', 'doc3576', 'doc3577', 'doc3578', 'doc3579', 'doc3580', 'doc3581', 'doc3582', 'doc3583', 'doc3584', 'doc3585', 'doc3586', 'doc3587', 'doc3588', 'doc3589', 'doc3590', 'doc3591', 'doc3592', 'doc3593', 'doc3594', 'doc3595', 'doc3596', 'doc3597', 'doc3598', 'doc3599', 'doc3600', 'doc3601', 'doc3602', 'doc3603', 'doc3604', 'doc3605', 'doc3606', 'doc3607', 'doc3608', 'doc3609', 'doc3610', 'doc3611', 'doc3612', 'doc3613', 'doc3614', 'doc3615', 'doc3616', 'doc3617', 'doc3618', 'doc3619', 'doc3620', 'doc3621', 'doc3622', 'doc3623', 'doc3624', 'doc3625', 'doc3626', 'doc3627', 'doc3628', 'doc3629', 'doc3630', 'doc3631', 'doc3632', 'doc3633', 'doc3634'

In [ ]:
for i in range(len(db_info['metadatas'])):
    if 'Указ Президента РФ от 30.03.2022 N 166 (ред. от 22.11.2023).pdf' in db_info['metadatas'][i]['source']:
        print('_______')
        print(db_info['documents'][i])

In [10]:
ids_ = []
metadatas_ = []
documents_ = []
embs_ = []
for i in range(len(db_info['metadatas'])):
    if 'gosts' not in db_info['metadatas'][i]['source']:
        ids_.append(db_info['ids'][i])
        documents_.append(db_info['documents'][i])
        embs_.append(db_info['embeddings'][i])
        metadatas_.append(db_info['metadatas'][i])



In [60]:
documents_strs = []
metadatas = []
for i in range(len(chunks)):
    for d in chunks[i]:
        documents_strs.append(d.page_content)
        metadatas.append(d.metadata)

ids = [f'doc{i}' for i in range(3926, len(metadatas) + 3926)]

In [46]:
documents_strs

['Приказ ФСТЭК России от 21.12.2017 N 235 (ред. от 20.04.2023) "Об утверждении Требований к созданию систем безопасности значимых объектов критической информационной инфраструктуры Российской Федерации и обеспечению их функционирования" (Зарегистрировано в Минюсте России 22.02.2018 N 50118)',
 'Зарегистрировано в Минюсте России 22 февраля 2018 г. N 50118 ФЕДЕРАЛЬНАЯ СЛУЖБА ПО ТЕХНИЧЕСКОМУ И ЭКСПОРТНОМУ КОНТРОЛЮ ПРИКАЗ от 21 декабря 2017 г. N 235 ОБ УТВЕРЖДЕНИИ ТРЕБОВАНИЙ К СОЗДАНИЮ СИСТЕМ БЕЗОПАСНОСТИ ЗНАЧИМЫХ ОБЪЕКТОВ КРИТИЧЕСКОЙ ИНФОРМАЦИОННОЙ ИНФРАСТРУКТУРЫ РОССИЙСКОЙ ФЕДЕРАЦИИ И ОБЕСПЕЧЕНИЮ ИХ ФУНКЦИОНИРОВАНИЯ Список изменяющих документов (в ред. Приказов ФСТЭК России от 27.03.2019 N 64 , от 20.04.2023 N 69 ) В соответствии с пунктом 4 части 3 статьи 6  Федерального закона от 26 июля 2017 г. N 187-ФЗ "О безопасности критической информационной инфраструктуры Российской Федерации" (Собрание законодательства Российской Федерации, 2017, N 31, ст. 4736) приказываю: Утвердить прилагаемые

In [51]:
import time
from gigachat.exceptions import ResponseError

embedded_docs = []
doc_idx_cant_process = []
for i in range(len(documents_strs)):
    try:
        emb = embedding_function_gigachat.embed_documents([documents_strs[i]])
        embedded_docs.append(emb)
    except ResponseError as ex:
        print(ex)
        doc_idx_cant_process.append(i)
        embedded_docs.append(-1)

    # if i % 10 == 0:
    #     time.sleep(1)


In [3]:
embedded_docs

NameError: name 'embedded_docs' is not defined

In [53]:
valid_docs = []
valid_embs = []
valid_metadatas = []
for i in range(len(embedded_docs)):
    if embedded_docs[i] == -1:
        pass
    else:
        valid_docs.append(documents_strs[i])
        valid_embs.append(embedded_docs[i])
        valid_metadatas.append(metadatas[i])

ids = [f'doc{i}' for i in range(len(db_info['ids']) - len(doc_idx_cant_process),
                                len(valid_docs) + len(db_info['ids']) - len(doc_idx_cant_process))]

In [54]:
valid_embs_ = []
for emb in valid_embs:
    valid_embs_.append(emb[0])

In [61]:
len(valid_docs), len(valid_embs_), len(valid_metadatas), len(ids)

(39, 39, 39, 39)

In [62]:
valid_metadatas

[{'source': '/Users/gd/Downloads/ФСТЭК_235.pdf', 'page': 0},
 {'source': '/Users/gd/Downloads/ФСТЭК_235.pdf', 'page': 1},
 {'source': '/Users/gd/Downloads/ФСТЭК_235.pdf', 'page': 1},
 {'source': '/Users/gd/Downloads/ФСТЭК_235.pdf', 'page': 2},
 {'source': '/Users/gd/Downloads/ФСТЭК_235.pdf', 'page': 2},
 {'source': '/Users/gd/Downloads/ФСТЭК_235.pdf', 'page': 2},
 {'source': '/Users/gd/Downloads/ФСТЭК_235.pdf', 'page': 3},
 {'source': '/Users/gd/Downloads/ФСТЭК_235.pdf', 'page': 3},
 {'source': '/Users/gd/Downloads/ФСТЭК_235.pdf', 'page': 3},
 {'source': '/Users/gd/Downloads/ФСТЭК_235.pdf', 'page': 4},
 {'source': '/Users/gd/Downloads/ФСТЭК_235.pdf', 'page': 4},
 {'source': '/Users/gd/Downloads/ФСТЭК_235.pdf', 'page': 4},
 {'source': '/Users/gd/Downloads/ФСТЭК_235.pdf', 'page': 4},
 {'source': '/Users/gd/Downloads/ФСТЭК_235.pdf', 'page': 5},
 {'source': '/Users/gd/Downloads/ФСТЭК_235.pdf', 'page': 5},
 {'source': '/Users/gd/Downloads/ФСТЭК_235.pdf', 'page': 5},
 {'source': '/Users/gd/D

In [64]:
#Возможно за раз не нужно добавлять сразу все документы, лучше (по частям)
chromadb_chunk_size_1200._collection.add(
    documents=valid_docs,
    embeddings=valid_embs_,
    ids=ids,
    metadatas=valid_metadatas
)

In [3]:
sources = []
for i in db_info['metadatas']:
    sources.append(i['source'])


NameError: name 'db_info' is not defined

In [47]:
gosts = []
cnt_ = 0
for p in list(set(sources)):
    if 'FZ' in p:
        print(p)



/Users/gd/PycharmProjects/altirix_systems_chatbot/fz/Ob_elektronnoj_podpisi_ot_06.04.2011_N_63_FZ_red._ot_14.07.2022_.pdf
/Users/gd/PycharmProjects/altirix_systems_chatbot/fz/Federal_nyy_zakon_ot_26.07.2017_N_187_FZ_O_bezopasnosti_kri.pdf
/Users/gd/PycharmProjects/altirix_systems_chatbot/gossopka_acts/194-FZ.pdf


In [43]:
cnt_

165

In [2]:
import datetime
print(datetime.datetime.now())

2025-02-11 18:03:41.057998


In [4]:
chromadb_chunk_size_1200 = Chroma(
    persist_directory="/Users/gd/PycharmProjects/altirix_systems_chatbot/chromadb_chunk_size_1200_fstek_cosine",
    embedding_function=embedding_function_gigachat)



In [5]:
info = chromadb_chunk_size_1200.get()

In [8]:
sources = []
for i in info['metadatas']:
    sources.append(i['source'])

In [13]:
list(set(sources))

['/Users/gd/PycharmProjects/altirix_systems_chatbot/Федеральная служба по техническому и экспортному контролю/МЕТОДИКА ОПРЕДЕЛЕНИЯ АКТУАЛЬНЫХ УГРОЗ БЕЗОПАСНОСТИ ПЕРСОНАЛЬНЫХ ДАННЫХ ПРИ ИХ ОБРАБОТКЕ В ИНФОРМАЦИОННЫХ СИСТЕМАХ ПЕРСОНАЛЬНЫХ ДАННЫХ.pdf',
 '/Users/gd/PycharmProjects/altirix_systems_chatbot/Федеральная служба по техническому и экспортному контролю/ФСТЭК приказ от 17.02.2017 № 27.pdf',
 '/Users/gd/PycharmProjects/altirix_systems_chatbot/Федеральная служба по техническому и экспортному контролю/ФСТЭК приказ от 11.02.2013 № 17(в ред. Приказа ФСТЭК России от 15.02.2017 № 27) .pdf',
 '/Users/gd/PycharmProjects/altirix_systems_chatbot/Федеральная служба по техническому и экспортному контролю/ФСТЭК приказ от 23.03.2017 № 49.pdf',
 '/Users/gd/PycharmProjects/altirix_systems_chatbot/Федеральная служба по техническому и экспортному контролю/ФСТЭК приказ от 25.12.2017 № 239 (в ред. Приказа ФСТЭК России от 09.08.2018 № 138).pdf',
 '/Users/gd/PycharmProjects/altirix_systems_chatbot/Федера